# Environement Setup

In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


Ambiente locale rilevato.


# Dataset Setup

In [2]:
from utils import setup_project
# Ora puoi passare agli import del modello
from dataset.capitain_cook_4d_mlp_dataset import CaptainCook4DMLP_Dataset, DatasetSource
from models.BaselineV2_Transformer import BaselineV2_Transformer
from dataset.utils import SplitType

# Esegue: Setup Dati (unzip/copy), Login WandB, Setup Device
device = setup_project.initialize(ROOT_DIR)

# Import wandb
import wandb

Setup Progetto in: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection
source_path: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
Setup Dati da: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
Inizio setup dati...
   Sorgente: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
   Destinazione: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data
Copia cartella: annotation_json...
Copia cartella: omnivore...
Estrazione ZIP: omnivore.zip...
✅ Setup completato! Dati pronti in: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\marco\_netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


WandB Logged in.
Device: cuda


# Configuration

In [ ]:
# Configurazione esperimento
DATASET_SOURCE = DatasetSource.HIERO

config = {
    "dataset": "CaptainCook4D",
    "feature_extractor": DATASET_SOURCE.value,
    "input_dim": DATASET_SOURCE.input_dims(),
    "batch_size": 1,  # DEVE essere 1 per sequenze di lunghezza variabile
    "learning_rate": 1e-5,
    "epochs": 30,
    "pos_weight": 0.75,
    "optimizer": "Adam",
    "loss_function": "BCEWithLogitsLoss",
    "seed": 42,
}

# Dataloader

In [9]:
from dataset.dagnn_dataset import DAGNNDataset

dataset = DAGNNDataset(
    video_embeddings_path=os.path.join(ROOT_DIR, "data", "hiero_all_video_steps.npz"),
    recipe_embeddings_dir=os.path.join(ROOT_DIR, "data", "recipe_embeddings"),
    hungarian_results_path=os.path.join(ROOT_DIR, "hungarian_results", "hungarian_matching_results.json"),
    annotation_path=os.path.join(ROOT_DIR, "data", "annotation_json", "video_level_annotations.json"),
)

Loading video embeddings...
Loading Hungarian matching results...
Loading error annotations...
Dataset initialized with 384 samples


In [11]:
dataset[0]

{'node_features': tensor([[-0.0103, -0.0238,  0.0335,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0272, -0.0033, -0.0309,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0357, -0.0621, -0.0214,  ...,  0.1494, -0.1945,  0.2338],
         ...,
         [-0.0034, -0.0594,  0.0373,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0383, -0.0292, -0.0189,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0742, -0.0109,  0.0238,  ...,  0.0000,  0.0000,  0.0000]]),
 'edge_index': tensor([[ 7, 10, 20,  9,  9,  5,  3, 21, 22,  8, 11, 18, 15, 13, 12,  1,  2,  4,
           6, 23, 14,  8, 16, 19,  0,  0,  0,  0,  0,  0,  0,  0, 16, 17],
         [ 2,  2,  2,  3,  5,  8,  8,  9,  9, 11, 13, 13, 14, 15, 16, 16, 16, 16,
          16, 16, 17, 18, 21, 23,  1,  4,  6,  7, 10, 12, 19, 20, 22, 24]]),
 'video_key': '16_39_360p_224.mp4_1s_1s',
 'recipe_name': 'scrambledeggs',
 'label': tensor(1),
 'match_mask': tensor([False, False,  True, False,  True, False, False,  True, False, False,
          True,  True,